In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from keras.layers import *
from keras import initializers, regularizers, constraints, optimizers, layers
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/glove6b50dtxt/glove.6B.50d.txt


In [2]:
import numpy as np 
import pandas as pd 
import nltk
import os
import gc
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import  Input,TimeDistributed, GRU, Dense,GaussianDropout,GaussianNoise,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout3D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
from string import punctuation
import re
from tqdm import tqdm
from tensorflow.keras.layers import Attention


In [3]:
from zipfile import ZipFile 

with ZipFile('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', 'r') as zip: 
    zip.extractall() 
with ZipFile('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', 'r') as zip: 
    zip.extractall()    

In [4]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
EMBEDDING_FILE=f'../input/glove840b300dtxt/glove.840B.300d.txt'


In [5]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values

In [6]:
train 

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [7]:
test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [8]:
train = train["comment_text"].str.lower()
# train2 = re.sub('[^a-zA-Z]',' ',train1)
test = test["comment_text"].str.lower()
# test = re.sub('[^a-zA-Z]',' ',test)


In [9]:
max_features = 50000
maxlen = 100

In [10]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train))
list_tokenized_train = tokenizer.texts_to_sequences(train)
list_tokenized_test = tokenizer.texts_to_sequences(test)
train = pad_sequences(list_tokenized_train, maxlen=maxlen)
test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [11]:
embed_size = 300

embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

2196018it [05:12, 7020.80it/s]


In [12]:
word_index = tokenizer.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [14]:
train

array([[    0,     0,     0, ...,  4583,  2273,   985],
       [    0,     0,     0, ...,   589,  8377,   182],
       [    0,     0,     0, ...,     1,   737,   468],
       ...,
       [    0,     0,     0, ...,  3509, 13675,  4528],
       [    0,     0,     0, ...,   151,    34,    11],
       [    0,     0,     0, ...,  1627,  2056,    88]], dtype=int32)

In [18]:
from keras.layers import Layer
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [39]:
inputs=Input((maxlen,))
x = Embedding(max_features, 300, weights=[embedding_matrix],trainable=False)(inputs)
x = LSTM(100,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x)
x = TimeDistributed(Dense(80))(x)
x = Dropout(0.2)(x)
x = attention()(x)
x = Dropout(0.2)(x)
outputs = Dense(6,activation='sigmoid',trainable=True)(x)
model = Model(inputs,outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 100)]             0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 100, 300)          15000000  
_________________________________________________________________
lstm_15 (LSTM)               (None, 100, 100)          160400    
_________________________________________________________________
time_distributed_13 (TimeDis (None, 100, 80)           8080      
_________________________________________________________________
dropout_16 (Dropout)         (None, 100, 80)           0         
_________________________________________________________________
attention_14 (attention)     (None, 80)                180       
_________________________________________________________________
dropout_17 (Dropout)         (None, 80)                0  

In [40]:
early_stopping =EarlyStopping(monitor='val_loss', patience=5)
model.fit(train, y,
        validation_split=0.1,
        epochs=2, batch_size=256, shuffle=True,
         callbacks=[early_stopping])

Epoch 1/2
561/561 [==============================] - 220s 393ms/step - loss: 0.0926 - accuracy: 0.7615 - val_loss: 0.0584 - val_accuracy: 0.9911
Epoch 2/2
561/561 [==============================] - 219s 391ms/step - loss: 0.0559 - accuracy: 0.9010 - val_loss: 0.0515 - val_accuracy: 0.9934


In [41]:
y_pred = model.predict(test, batch_size=1024)
print(y_pred)